In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle 
from sklearn import linear_model

### Load the train data 
- Load the train data from the xlsx file from dataset folder
- make timestamp as index

In [2]:
xlsx_path_train_data = "trainset.xlsx"
xlsx_path_test_data = "testset.xlsx"

# Read the CSV file into a pandas DataFrame
df_train = pd.read_excel(xlsx_path_train_data)
df_train["timestamp"] = pd.to_datetime(df_train["timestamp"], format="%Y-%m-%d %H:%M:%S")
df_train.set_index("timestamp", inplace=True)
# Print the first 5 rows of the DataFrame to make sure it was loaded correctly
print(df_train.head())

                     y_var    grade   x_var_1    x_var_2   x_var_3   x_var_4  \
timestamp                                                                      
2021-01-30 06:23:06     10  grade_1  34.30884   9.839399  64.52522  40.16026   
2021-01-30 10:09:38     25  grade_1  33.79903   9.861699  67.17016  32.47632   
2021-01-30 14:35:20      0  grade_1  33.68362  10.072020  66.39706  25.32029   
2021-01-30 19:46:07      5  grade_1  33.36954   9.795143  69.43247  27.25093   
2021-01-30 22:22:13      0  grade_1  34.64005   9.872020  68.75052  30.09163   

                      x_var_5    x_var_6    x_var_7   x_var_8  ...  x_var_110  \
timestamp                                                      ...              
2021-01-30 06:23:06  34.13535  10.200400   98.12772  44.70541  ...   0.295324   
2021-01-30 10:09:38  34.60092  10.663730  102.19990  45.31439  ...   0.318683   
2021-01-30 14:35:20  34.15429  10.617100  149.04180  56.12695  ...   0.299642   
2021-01-30 19:46:07  33.80330   9.

### Make a  feature data set for the training data 
- drop the unnecessary column 
- drop the target column 

In [3]:
# Get the column index of the 114th column (since we want to keep the first 114 columns)
drop_cols = list(df_train.columns[114:])

# Drop the columns from the DataFrame
df_train = df_train.drop(drop_cols, axis=1)

# Print the resulting DataFrame to verify that the columns were dropped
print(df_train.head())

                     y_var    grade   x_var_1    x_var_2   x_var_3   x_var_4  \
timestamp                                                                      
2021-01-30 06:23:06     10  grade_1  34.30884   9.839399  64.52522  40.16026   
2021-01-30 10:09:38     25  grade_1  33.79903   9.861699  67.17016  32.47632   
2021-01-30 14:35:20      0  grade_1  33.68362  10.072020  66.39706  25.32029   
2021-01-30 19:46:07      5  grade_1  33.36954   9.795143  69.43247  27.25093   
2021-01-30 22:22:13      0  grade_1  34.64005   9.872020  68.75052  30.09163   

                      x_var_5    x_var_6    x_var_7   x_var_8  ...  x_var_103  \
timestamp                                                      ...              
2021-01-30 06:23:06  34.13535  10.200400   98.12772  44.70541  ...   0.997892   
2021-01-30 10:09:38  34.60092  10.663730  102.19990  45.31439  ...   1.000515   
2021-01-30 14:35:20  34.15429  10.617100  149.04180  56.12695  ...   0.998131   
2021-01-30 19:46:07  33.80330   9.

##### change the data type of categorical variable

In [4]:

df_train["grade"] = df_train["grade"].astype("category")

#### Create a pipeline for preprocessing the Categorical and Numerical datatypes

In [5]:
num_cols = df_train.select_dtypes(include=["int", "float"]).columns
cat_cols = df_train.select_dtypes(include=["category"]).columns

# Remove the target column "y_var" from num_cols
num_cols = num_cols.drop("y_var")

# Create a pipeline for numerical features
num_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("scaler", StandardScaler())
])

# Create a pipeline for categorical features
cat_transformer = OneHotEncoder()

# Combine the numerical and categorical transformers into a single ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_cols),
        ("cat", cat_transformer, cat_cols)
    ])



### Fit the pipeline to Train dataset

In [6]:
# Fit the preprocessor to the data and transform it
X_preprocessed_train = preprocessor.fit_transform(df_train)

# Get the feature names from the preprocessor
cat_feature_names = preprocessor.named_transformers_["cat"].get_feature_names_out(cat_cols)
num_feature_names = list(num_cols)
num_scaled_feature_names = [f"{name}_scaled" for name in num_cols]
feature_names = num_scaled_feature_names + list(cat_feature_names)
# Create a new DataFrame with the preprocessed features
X_preprocessed_train = pd.DataFrame(X_preprocessed_train, columns=feature_names)

# Set the timestamp as the index for the X_preprocessed_train DataFrame
X_preprocessed_train.index = df_train.index

# Add the "y_var" target column to the X_preprocessed_train DataFrame
X_preprocessed_train["y_var"] = df_train["y_var"]

# Print the preprocessed DataFrame to verify that it was processed correctly
X_preprocessed_train



,x_var_1_scaled,x_var_2_scaled,x_var_3_scaled,x_var_4_scaled,x_var_5_scaled,x_var_6_scaled,x_var_7_scaled,x_var_8_scaled,x_var_9_scaled,x_var_10_scaled,...,x_var_107_scaled,x_var_108_scaled,x_var_109_scaled,x_var_110_scaled,x_var_111_scaled,x_var_112_scaled,grade_grade_1,grade_grade_2,grade_grade_3,y_var
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-30 06:23:06,0.639685,-0.037783,0.002424,1.232638,0.601569,0.478597,-0.903369,-0.868268,-0.677562,-0.803541,...,-0.236389,0.616214,0.953420,0.695781,0.320443,0.653325,1.0,0.0,0.0,10
2021-01-30 10:09:38,0.547081,-0.002072,0.730616,0.283144,0.684520,1.128354,-0.838183,-0.811933,-0.672928,-0.798678,...,-0.731750,-0.273826,0.450536,0.812186,0.320443,0.652192,1.0,0.0,0.0,25
2021-01-30 14:35:20,0.526118,0.334737,0.517770,-0.601117,0.604944,1.062962,-0.088360,0.188313,-0.640504,0.038557,...,-0.762677,0.009248,0.474578,0.717300,0.337591,0.636352,1.0,0.0,0.0,0
2021-01-30 19:46:07,0.469068,-0.108655,1.353464,-0.362550,0.542407,-0.148332,-0.871183,0.114424,-0.649769,0.035172,...,-0.686718,0.405553,0.341333,0.658835,0.337591,0.635960,1.0,0.0,0.0,5
2021-01-30 22:22:13,0.699847,0.014456,1.165713,-0.011529,0.621225,1.660144,-0.847844,0.121574,-0.642820,0.041626,...,-0.602620,0.608313,0.304942,0.690354,0.334677,0.650200,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-22 18:32:09,-1.855808,0.314351,-3.045927,-0.125610,-1.957121,0.212639,-0.863753,-1.408127,0.181708,-1.327299,...,-1.143557,-0.854986,-0.440516,-1.073748,-1.044577,-1.019861,0.0,1.0,0.0,5
2023-01-23 10:11:27,-2.984728,0.476606,-4.116398,0.017892,-1.891707,-0.020583,-0.884889,-1.228875,0.237297,-1.330143,...,-0.394818,-0.854986,-0.561671,-1.073748,-1.044577,-1.019861,0.0,1.0,0.0,5
2023-01-23 18:27:56,-2.727736,-0.428488,-4.059791,-0.706058,-1.743597,-0.173100,-0.884420,-1.152399,0.255825,-1.330133,...,-0.057343,0.088246,-0.286641,-1.073748,-1.044577,-1.019861,0.0,1.0,0.0,0


### Test set Preprocessing for model fitting 

In [7]:
# Read the CSV file into a pandas DataFrame
df_test = pd.read_excel(xlsx_path_test_data)
df_test["timestamp"] = pd.to_datetime(df_test["timestamp"], format="%Y-%m-%d %H:%M:%S")
df_test.set_index("timestamp", inplace=True)

# Print the first 5 rows of the DataFrame to make sure it was loaded correctly
df_test.head()

,grade,x_var_1,x_var_2,x_var_3,x_var_4,x_var_5,x_var_6,x_var_7,x_var_8,x_var_9,...,x_var_110,x_var_111,x_var_112,x_var_113,x_var_114,x_var_115,x_var_116,x_var_117,x_var_118,x_var_119
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-07-24 06:29:01,grade_3,35.01566,10.436260,66.43660,35.12374,35.18323,10.431910,148.9275,40.73342,0.254684,...,-0.045407,-0.173117,-0.189182,65.28326,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-24 11:05:59,grade_3,35.74119,10.325410,66.98230,33.11600,35.06469,10.625820,150.9064,41.22916,0.252195,...,-0.043682,-0.166538,-0.181992,65.31578,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-25 16:41:45,grade_1,26.72243,9.405282,62.53549,17.10056,25.12052,9.799699,201.5350,53.68760,0.160020,...,-0.062863,-0.239669,-0.261909,45.33643,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-25 17:34:41,grade_1,26.94120,10.003540,61.39827,30.00865,26.86813,10.024990,199.3965,53.15531,0.162700,...,-0.062863,-0.239669,-0.261909,22.65160,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-26 02:17:04,grade_1,26.45110,10.182280,62.19590,40.07494,26.37877,10.611760,199.9151,59.76392,0.152745,...,-0.062863,-0.227686,-0.248814,47.73375,NaN,NaN,NaN,NaN,NaN,NaN


#### Drop the unncessary columns from the test set

In [8]:
# Get the column index of the 114th column (since we want to keep the first 114 columns)
drop_cols = list(df_test.columns[114:])

# Drop the columns from the DataFrame
df_test = df_test.drop(drop_cols, axis=1)

In [9]:
df_test["grade"] = df_test["grade"].astype("category")

#### Fitting the test data to the  Preprocessing pipeline

In [10]:
# Fit the preprocessor to the data and transform it
X_preprocessed_test = preprocessor.transform(df_test)

# Get the feature names from the preprocessor
cat_feature_names = preprocessor.named_transformers_["cat"].get_feature_names_out(cat_cols)
num_feature_names = list(num_cols)
num_scaled_feature_names = [f"{name}_scaled" for name in num_cols]
feature_names = num_scaled_feature_names + list(cat_feature_names)
X_preprocessed_test = pd.DataFrame(X_preprocessed_test, columns=feature_names)
# Set the timestamp as the index for the X_preprocessed_train DataFrame
X_preprocessed_test.index = df_test.index
# Print the preprocessed DataFrame to verify that it was processed correctly
X_preprocessed_test.head()

,x_var_1_scaled,x_var_2_scaled,x_var_3_scaled,x_var_4_scaled,x_var_5_scaled,x_var_6_scaled,x_var_7_scaled,x_var_8_scaled,x_var_9_scaled,x_var_10_scaled,...,x_var_106_scaled,x_var_107_scaled,x_var_108_scaled,x_var_109_scaled,x_var_110_scaled,x_var_111_scaled,x_var_112_scaled,grade_grade_1,grade_grade_2,grade_grade_3
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-07-24 06:29:01,0.768074,0.918033,0.528656,0.610282,0.788271,0.803258,-0.090190,-1.235708,1.673273,-1.199076,...,-0.906648,1.017477,0.420036,0.437013,-1.002126,-1.000727,-0.962716,0.0,0.0,1.0
2021-07-24 11:05:59,0.899861,0.740517,0.678895,0.362188,0.767150,1.075191,-0.058512,-1.189848,1.613056,-1.204985,...,-0.492287,1.826442,-0.102665,0.894544,-0.993527,-0.995462,-0.955855,0.0,0.0,1.0
2021-07-25 16:41:45,-0.738332,-0.732980,-0.545378,-1.616818,-1.004611,-0.083331,0.751926,-0.037345,-0.617342,-0.121961,...,-0.323094,-0.210889,0.097462,0.390517,-1.089112,-1.053983,-1.032120,1.0,0.0,0.0
2021-07-25 17:34:41,-0.698594,0.225073,-0.858472,-0.021783,-0.693237,0.232609,0.717694,-0.086586,-0.552493,-0.115093,...,-0.308179,0.019701,1.077033,0.409930,-1.089112,-1.053983,-1.032120,1.0,0.0,0.0
2021-07-26 02:17:04,-0.787618,0.511308,-0.638873,1.222095,-0.780427,1.055473,0.725996,0.524761,-0.793365,0.499359,...,-0.439619,0.066362,0.310756,0.265157,-1.089112,-1.044394,-1.019623,1.0,0.0,0.0


# Model Training 

In [13]:
# Define the name of the pickle file
filename = 'model.pkl'

# Try to load the model from the pickle file
try:
    with open(filename, 'rb') as file:
        model = pickle.load(file)
except:
    # If the file doesn't exist or there's an error loading it, define a new model
    model = linear_model.Ridge(alpha=2)

# Iterate over the indices of the test DataFrame
predictions = []
for i in range(len(df_test.index)):
    # Get the index of the current test data point
    test_index = df_test.index[i]

    # Get the subset of the train DataFrame that occurs between the previous test index and the current test index
    if i == 0:
        train_subset = X_preprocessed_train.loc[:test_index]
    else:
        prev_test_index = df_test.index[i-1]
        train_subset = X_preprocessed_train.loc[prev_test_index:test_index]

    # Check if the train_subset is not empty
    if not train_subset.empty:
        # Separate the features and target variable
        X_train = train_subset.drop('y_var', axis=1)
        y_train = train_subset['y_var']

        # Update the model with the new training data
        model.fit(X_train, y_train)

        # Save the updated model as a pickle file
        with open(filename, 'wb') as file:
            pickle.dump(model, file)

    # Use the updated model to make predictions on the current test data point
    X_test = X_preprocessed_test.iloc[[i]]
    y_pred = model.predict(X_test)

    # Add the predicted value to the list of predictions
    predictions.append(y_pred[0])

In [14]:
print(predictions)

[31.66177170137273, 29.505254613645207, 10.800525659269418, 12.239869601525346, 34.92213230894694, 37.61917474448922, 37.43787425138653, 38.336347831131945, 38.77770399361008, 37.966726664072, 0.18522782803356408, 15.660670631266434, 2.894239665454843, 4.173212832289079, 2.7236365896550323, 3.7200045812090474, 7.619706876092648, 6.461528301148306, 0.0, 13.612409287871849, 4.683684933850546, 6.987070856461834, 20.0, 0.0, 0.0, 0.0, 11.020664047448104, 9.378341967493757, 5.0, 5.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 20.566225509147838, 19.422053402113953, 25.351784191963464, 24.94859557696543, -2.00015418389858, 10.656058117231703, 13.285579554231404, 17.22448841319396, 28.363073836785382, 21.87782901850207, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 10.0, 15.0, 15.0, 15.0, 25.0, 0.0, 20.0, 24.733453712080447, 20.11645821426991, 20.968048983286746, 5.0, 24.49057970362847, 30.97300031310108, -0.5888126206126891, 25.69246984685202, 30.29340461360571, 27.7278940289384

### Removing negative numbers and converting to nearest multiple of 5

In [15]:
# Removing negative numbers and converting to nearest multiple of 5
num_list = predictions
for i in range(len(num_list)):
    if num_list[i] < 0:
        num_list[i] = 0
    else:
        num_list[i] = (num_list[i] + 2) // 5 * 5
predictions = num_list

In [16]:
print(predictions)

[30.0, 30.0, 10.0, 10.0, 35.0, 35.0, 35.0, 40.0, 40.0, 35.0, 0.0, 15.0, 0.0, 5.0, 0.0, 5.0, 5.0, 5.0, 0.0, 15.0, 5.0, 5.0, 20.0, 0.0, 0.0, 0.0, 10.0, 10.0, 5.0, 5.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 20.0, 20.0, 25.0, 25.0, 0, 10.0, 15.0, 15.0, 30.0, 20.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 10.0, 15.0, 15.0, 15.0, 25.0, 0.0, 20.0, 25.0, 20.0, 20.0, 5.0, 25.0, 30.0, 0, 25.0, 30.0, 25.0, 0.0, 15.0, 20.0, 15.0, 15.0, 15.0, 15.0, 20.0, 75.0, 65.0, 55.0, 60.0, 45.0, 20.0, 15.0, 20.0, 15.0, 10.0, 10.0, 10.0, 10.0, 10.0, 5.0, 35.0, 5.0, 0.0, 5.0, 5.0, 5.0, 5.0, 0, 45.0, 95.0, 45.0, 55.0, 45.0, 10.0, 10.0, 0.0, 0.0, 5.0, 5.0, 5.0, 5.0, 5.0, 10.0, 10.0, 0.0, 0.0, 0.0, 0.0, 20.0, 20.0, 20.0, 20.0, 10.0, 5.0, 20.0, 20.0, 20.0, 20.0, 10.0, 20.0, 20.0, 10.0, 0.0, 5.0, 10.0, 10.0, 10.0, 10.0, 10.0, 15.0, 25.0, 25.0, 30.0, 5.0, 5.0, 5.0, 15.0, 15.0, 15.0, 5.0, 10.0, 0.0, 0.0, 10.0, 10.0, 15.0, 15.0, 15.0, 20.0, 20.0, 20.0, 20.0, 50.0, 50.0, 45.0, 45.0, 15.0, 10.0, 15

#### Developing Prediction Dataframe

In [17]:
test_timestamp = pd.read_excel(xlsx_path_test_data)['timestamp'].to_list()
testset_predictions = pd.DataFrame({'timestamp':test_timestamp,'y_var':predictions})

### Exporting into Excel file

In [18]:
testset_predictions.to_excel("pred_test_set.xlsx",index=False)